#### Random Graph Generation

In [1]:
import numpy as np
import random
import pickle
import os

nV=[150]                      #nv is the number of vertices in the graph.  
nC=[15]                      #cv is the number of clients for each facility
num_graphs=5

problem='set-cover'
modes=['skewed[1-10]']
da=['training','testing']
# mode='/skewed[1-6]'
# mode='/skewed[1,10]'
for nv in nV:
    for nc in nC:
        for mode in modes:
            os.system('mkdir'+' data/')
            os.system('mkdir'+' data/'+problem)
            os.system('mkdir'+' data/'+problem+'/'+mode)
            os.system('mkdir'+' data/'+problem+'/'+mode+'/V='+str(nv)+'-C='+str(nc))
            os.system('mkdir'+' data/'+problem+'/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/testing')
            os.system('mkdir'+' data/'+problem+'/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/training')
            print(nv,nc,mode)
            for d in da:
                path='data/'+problem+'/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/'+d+'/'
                for graph in range(num_graphs):
                    adj=np.zeros([nv*nc,3])
                    new_pairs=[]
                    for i in range(1,nv+1):
                        y=np.sort(random.sample(range(1,nv+1), nc))
                        for j in (y):
                            new_pairs.append((i,j))
                    new_pairs=np.array(new_pairs)


                    adj[:,0]=new_pairs[:,0] #Set
                    adj[:,1]=new_pairs[:,1] #Element
                    #adj[:,2]= random.choices([1,10], weights = [3, 6], k = nc*nv)
                    #adj[:,2]np.random.randint(low = 1,high=6, size=nc*nv)
                    adj[:,2]= 1
                    #random.choices([1,2,3,4,5,6,7,8,9,10], weights = [1,1,2,2,3,3,4,4,5,5], k = nc*nv)
                    #adj[:,2]= random.choices([1,100], k = nc*nv)

                    os.system("mkdir "+path+str(graph)); pickle.dump([adj,nv],open(path+str(graph)+'/data','wb'))
                    np.savetxt(path+str(graph)+'/data.txt', adj, fmt='%10d' ,header=str(nv)+'   '+str(nv*nc))

150 15 skewed[1-10]


#### LP for feature computation

In [2]:
import xpress as xp
from termcolor import colored
import timeit
import numpy as np
import pickle
import random
def kmedian_xpress_solver_LP(set_path,c,n):
    print("Inside kmedian",n)
    S1 = timeit.default_timer()
    x = np.array ([xp.var (vartype = xp.continuous,ub=1.0) for i in range(n*n)]).reshape (n,n)
    y = np.array([xp.var (vartype = xp.continuous,ub=1.0) for i in range(n)])
    #x = np.array ([xp.var (vartype = xp.free) for i in range(n*n)]).reshape (n,n)
    #y = np.array([xp.var (vartype = xp.free) for i in range(n)])
    print(x.shape,y.shape)
    print(colored("----------",'blue'))
    print(c)
    c2 = [x[i,j] - y[i] <=0 for i in range(n) for j in range(n)]
    c3 = [xp.Sum(xp.Dot(x[:,j],c[:,j])) ==1 for j in range(n)]
    #c2=xp.Dot(x,np.ones([n,1]))==np.ones([n,1])
    #c3=xp.Dot(np.concatenate([-np.ones([n,1]),np.identity(n)],axis=1),np.concatenate([y.reshape(1,n),x],axis=0))<=np.zeros([n,n])

    #c4=0<=x; c5=x<=1;  c6=0<=y;  c7=y<=1

    profit=xp.Sum(xp.Sum(y))

    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.minimize)
    try:
        p.addConstraint(c2,c3)#,c4,c5,c6,c7)
    except xp.ModelError as e:
      print ("Modeling error:", repr(e))
    p.solve()
    y=p.getSolution(y)
    x=p.getSolution(x)
    E1 = timeit.default_timer()
    #################################################################################################
    median_indices=np.where(y>0.0)[0]
    print(colored('            Time: ','red'), E1 - S1) 
    print(colored('            objective value: ','red'),p.getSolution(profit))
    #print(colored('median indices: ','red'),median_indices+1,median_indices.shape)
     #------------------------------------------------------
    pickle.dump([median_indices,p.getSolution(profit),E1-S1],open(set_path+'/lp_inf','wb'))
    return median_indices,x,y

#### Computing LP based features

In [3]:
def features_related_LP(set_path,median_indices,x,y,c,n):
    #------------------lp_kmedian----------------------
    ind=np.zeros(n);ind[median_indices]=1
    pickle.dump(ind,open(set_path+'/lp_kmedian','wb'))
    pickle.dump(y,open(set_path+'/yf','wb'))
    #--------------------alfa_f-------------------------
    alfa_f=np.zeros(n)
    for i in range(n):
        alfa_f[i]= sum(x[:,i])
    pickle.dump(alfa_f,open(set_path+'/alfa_f','wb'))
    #---------------------µf-------------------------
    µf=np.zeros(n);
    for i in range(n):
        if alfa_f[i]!=0:
            µf[i]= sum(x[:,i]*c[:,i])/alfa_f[i]
    pickle.dump(µf,open(set_path+'/µf','wb'))
    #---------------------Vf,t(t=2,4,8)-------------------------
    T=[2,4,8]
    for t in T:
        V_ft=np.zeros(n);
        for i in range(n):
            V_ft[i]=len(np.where(c[:,i]<=(t*µf[i]))[0])
        pickle.dump(V_ft,open(set_path+'/V_f_'+str(t),'wb'))
    #---------------------Bf,t(t=2,4,8)-------------------------
    dav=np.zeros(n)
    for i in range(n):
        dav[i]=sum(c[:,i]*x[:,i])

    T=[2,4,8]
    for t in T:
        B_ft=np.zeros(n)
        for i in range(n):
            B_ft[i]=len(np.where(c[:,i]<=(t*dav[i]))[0])
        pickle.dump(B_ft,open(set_path+'/B_f_'+str(t),'wb'))
                    
    #----------------------------Yf----------------------------
    Yf=np.zeros(n)
    for i in range(n):
        Yf[i]=(len(np.where(x[:,i]!=0)[0]))
    pickle.dump(Yf,open(set_path+'/Yf','wb'))

#### Computing features not based on LP

In [4]:
def features_no_related_LP(Mypath,cost_mtr,nv):
    #--------------------------------------------------------
    degree=np.zeros(nv)
    for j in range(nv):
        s=np.where(cost_mtr[:,j]!=0)[0]
        degree[j]=sum(cost_mtr[s,j])
    pickle.dump(degree,open(Mypath+'/degree','wb'))
    #--------------------------------------------------------
    T=[2,4,8]
    for t in T:
        T_ft=np.zeros(nv);
        for i in range(nv):
            T_ft[i]=len(np.where(cost_mtr[:,i]<=t)[0])

        pickle.dump(T_ft,open(Mypath+'/T_f_'+str(t),'wb'))
    #--------------------------------------------------------
    min_c=np.zeros(nv)
    for i in range(nv):
        min_c[i]=min(cost_mtr[:,i])
    pickle.dump(min_c,open(Mypath+'/min_c','wb'))
    #---------------------------min_f-----------------------------
    min_f=np.zeros(nv)
    for i in range(nv):
        min_f[i]=min(cost_mtr[:,i])
    pickle.dump(min_f,open(Mypath+'/min_f','wb'))
    #----------------------------Yf,p=1.5----------------------------
    Yf=np.zeros(nv)
    for i in range(nv):
        Yf[i]=len(np.where(cost_mtr[:,i]<=(1.5*min_f[i]))[0])
    pickle.dump(Yf,open(Mypath+'/Yfp','wb'))
    #------------------------------------------------------------------------------------
    s=np.sort(random.sample(range(nv), int(nv/2)))
    s1=np.zeros(nv);s1[s]=1
    ss=[]
    for i in range(nv):
        if i not in s:
            ss.append(i)
    ss=np.array(ss);s2=np.zeros(nv);s2[ss]=1
    pickle.dump(s1,open(Mypath+'/f1','wb'))
    pickle.dump(s2,open(Mypath+'/f2','wb'))

#### Calling feature computation (LP-based and non LP-based) after computing LP

In [5]:
for Mode in modes:
    print(Mode,'--------------------------')
    for nv in nV:
        for nc in nC:
            print('------------------------------------------')
            for d in da:
                print('--- nV=',nv,' nC=',nc, ' mode=',d)
                for graph in range(num_graphs):
                    print('       data=',graph)
                    Mypath='data/'+problem+'/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/'+d+'/'+str(graph)
                    data = pickle.load(open(Mypath+'/data','rb'))
                    nv= data[1]; data=data[0]
                    data=data.astype(int)
                    print("Data: ",data)
                    data[:,0]=data[:,0]-1
                    data[:,1]=data[:,1]-1
                    cost_mtr=np.zeros([nv,nv])
                    for i in range (len(data)):
                        cost_mtr[data[i,0],data[i,1]]=data[i,2]
                    #print("Calling kmedian with ",Mypath,data.shape,cost_mtr.shape,nv,k)
                    median_indices,x,y=kmedian_xpress_solver_LP(Mypath,cost_mtr,nv)
                    features_related_LP(Mypath,median_indices,x,y,cost_mtr,nv)
                    features_no_related_LP(Mypath,cost_mtr,nv)

skewed[1-10] --------------------------
------------------------------------------
--- nV= 150  nC= 15  mode= training
       data= 0
Data:  [[  1  39   1]
 [  1  45   1]
 [  1  53   1]
 ...
 [150 105   1]
 [150 124   1]
 [150 127   1]]
Inside kmedian 150
(150, 150) (150,)
----------
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
FICO Xpress v8.10.0, Hyper, solve started 10:24:52, Aug 14, 2021
Heap usage: 10MB (peak 10MB, 1073KB system)
Minimizing LP noname
Original problem has:
     22650 rows        22650 cols        47250 elements
Presolved problem has:
      2400 rows         2400 cols         6750 elements
Presolve finished in 0 seconds
Heap usage: 11MB (peak 21MB, 1074KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  

--- nV= 150  nC= 15  mode= testing
       data= 0
Data:  [[  1   9   1]
 [  1  12   1]
 [  1  22   1]
 ...
 [150 141   1]
 [150 142   1]
 [150 149   1]]
Inside kmedian 150
(150, 150) (150,)
----------
[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
FICO Xpress v8.10.0, Hyper, solve started 10:24:54, Aug 14, 2021
Heap usage: 10MB (peak 10MB, 1111KB system)
Minimizing LP noname
Original problem has:
     22650 rows        22650 cols        47250 elements
Presolved problem has:
      2400 rows         2400 cols         6750 elements
Presolve finished in 0 seconds
Heap usage: 11MB (peak 21MB, 1112KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 1.00e+00,  1.00e+0

  2761         11.313001      P      0     0        .000000     0
Uncrunching matrix
Optimal solution found
Dual solved problem
  2761 simplex iterations in 0s

Final objective                       : 1.131300135633451e+01
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max dual violation        (abs/rel) :       0.0 /       0.0
  Max complementarity viol. (abs/rel) :       0.0 /       0.0
            Time:  0.40672800317406654
            objective value:  11.313001356334503



#### Run ILP to get labelled training data and evaluation on test data

In [6]:
import xpress as xp
from termcolor import colored
import timeit
import math
import numpy as np
import pickle
import sys
import matplotlib.pyplot as plt
import networkx as nx

def floyd_warshall(G,nV):
    distance = list(map(lambda i: list(map(lambda j: j, i)), G))

    # Adding vertices individually
    for k in range(nV):
        for i in range(nV):
            for j in range(nV):
                distance[i][j] = min(distance[i][j], distance[i][k] + distance[k][j])
    return (distance)

def kmedian_xpress_solver_ILP(c,n):
    x = np.array ([xp.var (vartype = xp.binary) for i in range(n*n)]).reshape (n,n)
    y = np.array([xp.var (vartype = xp.binary) for i in range(n)])
    #x = np.array ([xp.var (vartype = xp.free) for i in range(n*n)]).reshape (n,n)
    #y = np.array([xp.var (vartype = xp.free) for i in range(n)])
    print(x.shape,y.shape)
    print(colored("----------",'blue'))
    print(c)
    c2 = [x[i,j] - y[i] <=0 for i in range(n) for j in range(n)]
    c3 = [xp.Sum(xp.Dot(x[:,j],c[:,j])) ==1 for j in range(n)]
    #c2=xp.Dot(x,np.ones([n,1]))==np.ones([n,1])
    #c3=xp.Dot(np.concatenate([-np.ones([n,1]),np.identity(n)],axis=1),np.concatenate([y.reshape(1,n),x],axis=0))<=np.zeros([n,n])

    #c4=0<=x; c5=x<=1;  c6=0<=y;  c7=y<=1

    profit=xp.Sum(xp.Sum(y))

    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.minimize)
    try:
        p.addConstraint(c2,c3)#,c4,c5,c6,c7)
    except xp.ModelError as e:
      print ("Modeling error:", repr(e))
    p.solve()
    

    median_indices=np.where(p.getSolution(y)>=.5)[0]
    return median_indices,p.getSolution(profit)


for nv in nV:
    for nc in nC:
        for mode in modes:
            for d in da:
                path='data/'+problem+'/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/'+d+'/'
                for m in range(num_graphs):
                    print(colored('....................................................','blue'),m)
                    S = timeit.default_timer()
                    set_path=path+str(m)
                    data = pickle.load(open(set_path+'/data','rb'))
                    #print(data[0],data[1],data[2])
                    n = data[1]
                    data=data[0]
                    data=data.astype(int)
                    print("Data: ",data)
                    data[:,0]=data[:,0]-1
                    data[:,1]=data[:,1]-1
                    cost_mtr=np.zeros([n,n])
                    for i in range (len(data)):
                        cost_mtr[data[i,0],data[i,1]]=data[i,2]

                    labels=kmedian_xpress_solver_ILP(cost_mtr,n)

                    E = timeit.default_timer() 
                    pickle.dump([labels,E-S],open(set_path+'/points_inf_xp','wb'))

                    print(data[1],data[0].shape)
                    print(colored('objective value: ','red'),labels[1])
                    print(E - S) 
                    print(colored('median indices: ','red'),labels[0]+1,len(labels[0]))

.................................................... 0
Data:  [[  1  39   1]
 [  1  45   1]
 [  1  53   1]
 ...
 [150 105   1]
 [150 124   1]
 [150 127   1]]
(150, 150) (150,)
----------
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
FICO Xpress v8.10.0, Hyper, solve started 10:24:57, Aug 14, 2021
Heap usage: 11MB (peak 11MB, 1122KB system)
Minimizing MILP noname
Original problem has:
     22650 rows        22650 cols        47250 elements     22650 globals
Presolved problem has:
      2400 rows         2400 cols         6750 elements      2400 globals
Presolve finished in 0 seconds
Heap usage: 14MB (peak 28MB, 1124KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max

 
Starting root cutting & heuristics
 
 Its Type    BestSoln    BestBound   Sols    Add    Del     Gap     GInf   Time
   1  K     17.000000    11.920902      6      7      0   29.88%    1368     19
   2  K     17.000000    11.920902      6      4      5   29.88%    1297     19
   3  K     17.000000    11.920902      6      4      3   29.88%    1365     19
   4  K     17.000000    11.920902      6      3      1   29.88%    1352     20
   5  K     17.000000    11.920902      6      3      3   29.88%    1351     20
   6  K     17.000000    11.920902      6      2      1   29.88%    1358     21
   7  K     17.000000    11.920902      6      2      2   29.88%    1346     21
   8  K     17.000000    11.920902      6      2      2   29.88%    1369     22
   9  K     17.000000    11.920902      6      3      3   29.88%    1399     23
  10  K     17.000000    11.920902      6      3      2   29.88%    1398     23
  11  K     17.000000    11.920902      6      0      0   29.88%    1400     24
H

Heap usage: 941MB (peak 941MB, 75MB system)
B&B tree size: 27MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    9344    17.000000    11.920902      6   4503     16   29.88%     865     55
    9444    17.000000    11.920902      6   4552     18   29.88%     702     55
    9544    17.000000    11.920902      6   4603     28   29.88%     579     55
    9644    17.000000    11.920902      6   4609     37   29.88%     519     55
    9744    17.000000    11.920902      6   4711     36   29.88%     879     55
    9844    17.000000    11.920902      6   4734     30   29.88%     673     56
    9946    17.000000    11.920902      6   4735     22   29.88%     806     56
   10047    17.000000    11.920902      6   4786     42   29.88%     581     56
   10148    17.000000    11.920902      6   4786     48   29.88%     531     56
   10251    17.000000    11.920902      6   4803     31   29.88%     539     56
   10353    17.000000    11.920902      6   4845

   78159    16.000000    13.091347      7  24641     18   18.18%     675    134
   79164    16.000000    13.097418      7  24879     28   18.14%     624    135
   80164    16.000000    13.131331      7  25107     20   17.93%     727    136
   81164    16.000000    13.201171      7  25325     23   17.49%     868    137
   82164    16.000000    13.215924      7  25560     23   17.40%     622    138
   83168    16.000000    13.262402      7  25743     20   17.11%     649    139
   84173    16.000000    13.347369      7  25949     28   16.58%     808    140
Heap usage: 1111MB (peak 1111MB, 75MB system)
B&B tree size: 197MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
   85175    16.000000    13.361265      7  26142     24   16.49%     769    141
   86175    16.000000    13.387679      7  26353     26   16.33%     633    142
   87179    16.000000    13.387679      7  26552     22   16.33%     741    143
   88181    16.000000    13.403669      7  26

q           21.000000    11.413864      4                 45.65%       0      0
   1  K     21.000000    11.495625      4      8      0   45.26%    1201      0
   2  K     21.000000    11.531317      4      5      6   45.09%    1258      0
   3  K     21.000000    11.543706      4      3      3   45.03%    1273      0
   4  K     21.000000    11.560760      4      3      3   44.95%    1273      1
   5  K     21.000000    11.566216      4      3      2   44.92%    1264      1
   6  K     21.000000    11.568718      4      3      2   44.91%    1266      1
   7  K     21.000000    11.571262      4      3      1   44.90%    1283      2
   8  K     21.000000    11.575043      4      1      1   44.88%    1329      2
   9  K     21.000000    11.575100      4      4      5   44.88%    1294      3
  10  K     21.000000    11.575114      4      3      1   44.88%    1324      4
  11  K     21.000000    11.575150      4      3      3   44.88%    1319      4
  12  K     21.000000    11.575164      

...

  233644    16.000000    14.188250     11  24746     26   11.32%     634    265
  243644    16.000000    14.212987     11  23875     26   11.17%     608    273
  253644    16.000000    14.240188     11  22789     27   11.00%     595    282
  263644    16.000000    14.268978     11  21581     35   10.82%     552    290
  273644    16.000000    14.297979     11  20266     18   10.64%     639    298
  283645    16.000000    14.327000     11  18728     24   10.46%     664    306
  293645    16.000000    14.362972     11  17035     28   10.23%     602    314
  303645    16.000000    14.399071     11  15071     29   10.01%     575    322
  313647    16.000000    14.440283     11  12662     24    9.75%     666    330
  323647    16.000000    14.503450     11   9631     42    9.35%     647    338
  333647    16.000000    14.604403     11   5280     27    8.72%     727    347
 *** Search completed ***
Uncrunching matrix
Final MIP objective                   : 1.600000000000000e+01
Final MIP bou

    1700    19.000000    11.998491      5   1569      9   36.85%     977     21
    1800    19.000000    11.998491      5   1605     23   36.85%     857     21
    1901    19.000000    11.998491      5   1633    126   36.85%     396     22
    2001    19.000000    11.998491      5   1649    113   36.85%     418     22
    2102    19.000000    11.998491      5   1735     31   36.85%     768     22
Heap usage: 455MB (peak 455MB, 38MB system)
B&B tree size: 10MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    2202    19.000000    12.016396      5   1820     32   36.76%     754     22
    2303    19.000000    12.016396      5   1848     21   36.76%     732     23
    2404    19.000000    12.016396      5   1953     54   36.76%     531     23
    2505    19.000000    12.016396      5   2031     13   36.76%     965     23
    2608    19.000000    12.016396      5   2057     54   36.76%     454     23
    2708    19.000000    12.016396      5   2166

    7734    18.000000    12.016396      6   2746     47   33.24%     311     49
    7837    18.000000    12.016396      6   2913     71   33.24%     457     49
    7937    18.000000    12.016396      6   3073     78   33.24%     393     49
    8037    18.000000    12.016396      6   3156     47   33.24%     448     49
    8140    18.000000    12.016396      6   3208     38   33.24%     472     49
    8242    18.000000    12.016396      6   3356     32   33.24%     436     49
    8345    18.000000    12.016396      6   3460     38   33.24%     405     50
    8449    18.000000    12.016396      6   3575     46   33.24%     447     50
    8550    18.000000    12.016396      6   3706     55   33.24%     487     50
    8653    18.000000    12.016396      6   3742     50   33.24%     566     50
    8753    18.000000    12.016396      6   3777     48   33.24%     464     50
    8854    18.000000    12.016396      6   3805     53   33.24%     551     50
    8954    18.000000    12.016396      

   64335    17.000000    12.419332      7  34158     72   26.95%     575    112
   65335    17.000000    12.419332      7  34758     31   26.95%     687    113
   66336    17.000000    12.475471      7  35347     29   26.61%     619    115
Heap usage: 1158MB (peak 1158MB, 75MB system)
B&B tree size: 218MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
   67339    17.000000    12.475471      7  35924     45   26.61%     480    116
   68340    17.000000    12.507517      7  36464     37   26.43%     546    117
   69340    17.000000    12.507517      7  36958     37   26.43%     564    118
   70342    17.000000    12.507517      7  37715     34   26.43%     634    119
   71345    17.000000    12.591255      7  38108     34   25.93%     447    120
   72346    17.000000    12.591255      7  38781     39   25.93%     468    121
   73346    17.000000    12.591255      7  39206     17   25.93%     656    122
   74348    17.000000    12.592216      7  39

  694169    16.000000    14.520412      8  52813     27    9.25%     643    616
  704169    16.000000    14.547324      8  49164     32    9.08%     675    624
  714170    16.000000    14.578684      8  44924     29    8.88%     656    633
  724170    16.000000    14.618277      8  40059     39    8.64%     653    641
  734171    16.000000    14.663122      8  34466     42    8.36%     695    649
  744173    16.000000    14.721343      8  27894     29    7.99%     637    657
  754173    16.000000    14.796634      8  20201     19    7.52%     684    662
  764173    16.000000    14.876695      8  11597     42    7.02%     671    667
  774173    16.000000    14.971911      8   2291     35    6.43%     547    670
 *** Search completed ***
Uncrunching matrix
Final MIP objective                   : 1.600000000000000e+01
Final MIP bound                       : 1.600000000000000e+01
  Solution time / primaldual integral :       671s/ 17.603284%
  Number of solutions found / nodes   :         

#### Load libraries to run Classification model

In [7]:
import numpy as np
import sys
import pickle
import re
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from termcolor import colored
import xpress as xp
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,precision_recall_curve, confusion_matrix, accuracy_score
import xgboost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_curve
from sklearn import svm

#### Load features for classification

In [8]:
def feature_loader(set_path): 
    f0=pickle.load(open(set_path+'/alfa_f','rb'))
    f1=pickle.load(open(set_path+'/B_f_2','rb'))
    f2=pickle.load(open(set_path+'/B_f_4','rb'))
    f3=pickle.load(open(set_path+'/B_f_8','rb'))
    f4=pickle.load(open(set_path+'/lp_kmedian','rb'))
    f5=pickle.load(open(set_path+'/min_f','rb'))
    f6=pickle.load(open(set_path+'/V_f_2','rb'))
    f7=pickle.load(open(set_path+'/V_f_4','rb'))
    f8=pickle.load(open(set_path+'/V_f_8','rb'))
    f9=pickle.load(open(set_path+'/yf','rb'))
    f10=pickle.load(open(set_path+'/Yf','rb'))
    f11=pickle.load(open(set_path+'/µf','rb'))
    f12=pickle.load(open(set_path+'/Yfp','rb'))
#     f13=pickle.load(open(set_path+'/degree','rb'))
    f14=pickle.load(open(set_path+'/T_f_2','rb'))
    f15=pickle.load(open(set_path+'/T_f_4','rb'))
    f16=pickle.load(open(set_path+'/T_f_8','rb'))
#     f17=pickle.load(open(set_path+'/min_c','rb'))
#     f18=pickle.load(open(set_path+'/f1','rb'))
#     f19=pickle.load(open(set_path+'/f2','rb'))
#     f20=pickle.load(open(set_path+'/lp_prime','rb'))
#     f21=pickle.load(open(set_path+'/NLP1','rb'))
    #f22=pickle.load(open(set_path+'/NILP','rb'))
    #f23= pickle.load(open(set_path+'/ga_ind','rb'))
    
    features=np.vstack([f0[:],f1[:],f2[:],f3[:],f4[:],f5[:],f6[:],f7[:],f8[:],f9[:],f10[:],f11[:],f12[:],f14[:],f15[:],f16[:]]);features=features.transpose()
    #features=np.vstack([f0[:],f1[:],f2[:],f3[:],f4[:],f5[:],f6[:],f7[:],f8[:],f9[:],f10[:],f11[:]]);features=features.transpose()
    #features=np.vstack([f4[:],f10[:],f7[:],f2[:]]);features=features.transpose()
    
    
    return(features)

#### Load ground truth for classification

In [9]:
path='data/'+problem+'/'+modes[0]+'/V='+str(nV[0])+'-C='+str(nC[0])+'/training/0'
y=pickle.load(open(path+'/points_inf_xp','rb'))
y_true=np.zeros(nV[0]);y_true[y[0][0]]=1
features=feature_loader(path)


for Mode in modes:
    for nv in nV:
        for nc in nC:
            for i in range(1,num_graphs):
                Mypath='data/'+problem+'/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/training/'+str(i)
                y=pickle.load(open(Mypath+'/points_inf_xp','rb'))
                y_temp=np.zeros(nV[0]);y_temp[y[0][0]]=1
                y_true=np.hstack([y_true,y_temp])
                f_temp=feature_loader(Mypath)
                features=np.vstack([features,f_temp])
print(len(y_true),len(features))


750 750


#### Normalisation and Learning Classification Model

In [10]:
sc = StandardScaler()
features = sc.fit_transform(features)


clf = RandomForestClassifier(max_depth=5,n_estimators=100,min_samples_split=10, random_state=0,class_weight="balanced")
# clf = xgboost.XGBClassifier(n_estimators=100, learning_rate=1.0, max_depth=3, random_state=0, class_weight='balanced')
# clf=AdaBoostClassifier(n_estimators=100, random_state=0)
# clf=svm.SVC(cache_size=200, class_weight='balanced',random_state=None)

clf.fit(features,y_true)

print(colored('Feature importance:','blue'))
print(clf.feature_importances_)

Feature importance:
[0.13768344 0.         0.         0.         0.05417783 0.
 0.0835645  0.07731411 0.0755135  0.24479497 0.09911601 0.16110112
 0.06673452 0.         0.         0.        ]


#### Output the learnt model to a file

In [11]:
for Mode in modes:
    for nv in nV:
        for nc in nC:
            for i in range(num_graphs):
                test_path = 'data/'+problem+'/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/testing/'+str(i)

                test_features=feature_loader(test_path)
                test_features = sc.transform(test_features)

                y_pred = clf.predict(test_features)

                probs= clf.predict_proba(test_features)

                pickle.dump(np.where(y_pred==1)[0],open(test_path+'/pruning_points_'+str(i),'wb'))
                pickle.dump(probs[:,1],open(test_path+'/probs','wb'))
                pickle.dump(np.where(y_pred==1)[0],open(test_path+'/probs_xgb','wb'))


                print(len(np.where(y_pred==1)[0]))

36
39
38
35
38


#### Compute the ILP on pruned data set (pruning based on different thresholds)

In [12]:
import numpy as np
import sys
import pickle
import re
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from termcolor import colored
import xpress as xp
from sklearn.metrics import confusion_matrix
import random


def kmedian_xpress_solver_pruned(c,n,num_fac):
    x = np.array ([xp.var (vartype = xp.binary) for i in range(num_fac*n)]).reshape (num_fac,n)
    y = np.array([xp.var (vartype = xp.binary) for i in range(num_fac)])
    #x = np.array ([xp.var (vartype = xp.free) for i in range(n*n)]).reshape (n,n)
    #y = np.array([xp.var (vartype = xp.free) for i in range(n)])
    print(x.shape,y.shape)
    print(colored("----------",'blue'))
    print(c)
    c2 = [x[i,j] - y[i] <=0 for i in range(num_fac) for j in range(n)]
    c3 = [xp.Sum(xp.Dot(x[:,j],c[:,j])) ==1 for j in range(n)]
    #c2=xp.Dot(x,np.ones([n,1]))==np.ones([n,1])
    #c3=xp.Dot(np.concatenate([-np.ones([n,1]),np.identity(n)],axis=1),np.concatenate([y.reshape(1,n),x],axis=0))<=np.zeros([n,n])

    #c4=0<=x; c5=x<=1;  c6=0<=y;  c7=y<=1

    profit=xp.Sum(y)

    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.minimize)
    try:
        p.addConstraint(c2,c3)#,c4,c5,c6,c7)
    except xp.ModelError as e:
      print ("Modeling error:", repr(e))
    p.solve()
    
    
    median_indices=np.where(p.getSolution(y)>=0.5)[0]
    #print(p.getSolution(y),k)
    #print(median_indices)
    
    return median_indices,p.getSolution(profit)


#///////////////////////////////////////////////////////////////////////////////////////////////////
for nv in nV:
    for nc in nC:
        for Mode in modes:
            path='data/'+problem+'/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/testing/'
            thre = [max(int(nv/10),1),max(int(nv/5),1),max(int(3*nv/10),1),max(int(4*nv/10),1),int(nv/2),int(nv)]
            value_n=np.zeros(len(thre))
            Time=np.zeros(len(thre))
            for m in range(num_graphs):
                print(colored('....................................................','blue'),m)
                set_path=path+str(m)
                data = pickle.load(open(set_path+'/data','rb'))

                probs=pickle.load(open(set_path+'/probs','rb'))
                ind=np.argsort(probs)


                n=data[1];
                data=data[0].astype(int)
                data[:,0]=data[:,0]-1
                data[:,1]=data[:,1]-1

                for th_index,th in enumerate(thre):
                    ind_last_thre = ind[len(ind)-th:]
                    num_fac = len(ind_last_thre)

                    c=np.zeros([len(ind_last_thre),n])
                    for i in range(n):
                        for j in range(len(ind_last_thre)):
                                c[j,i]=0

                    for i in range (len(ind_last_thre)):
                        mp=(np.where(data[:,0]==ind_last_thre[i])[0])
                        c[i,data[mp,1]]=data[mp,2]

                    #print(colored("--------------------",'blue'))
                    #print(data,n,c,num_fac,k)
                    #print(colored("--------------------",'blue'))
                    S=timer()
                    median_indices,value=kmedian_xpress_solver_pruned(c,n,len(ind_last_thre))
                    E=timer()
                    print(colored('pruned centers:','red'),ind_last_thre+1)
                    print(colored('selected centers:','red'),np.sort(ind_last_thre[(median_indices)])+1,len(median_indices))
                    print(colored('Objective Value:','red'),value)
                    print(E-S)

                    s=pickle.load(open(set_path+'/points_inf_xp','rb'))
                    s=s[0][1]
                    value_n[th_index]=value/s
                    Time[th_index]=E-S
                    #print(s,value_n[thi],value)
                print(colored("Value_n: " +str(value_n),'red'))
                print(colored("Time: "+str(Time),'red'))

.................................................... 0
(15, 150) (15,)
----------
[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
FICO Xpress v8.10.0, Hyper, solve started 14:58:30, Aug 14, 2021
Heap usage: 1306KB (peak 1306KB, 1086KB system)
Minimizing MILP noname
Original problem has:
      2400 rows         2265 cols         4725 elements      2265 globals
 
 
The problem is infeasible due to row R455251
Presolve finished in 0 seconds
Heap usage: 1195KB (peak 2261KB, 1088KB system)
 *** Search completed ***
Problem is integer infeasible
  Solution time / primaldual integral :         0s/ 100.000000%
  Number of solutions found / nodes   :         0 /         0

pruned centers: [109  30  96 103  46  24  43  37 116  57  49 119 120  22  15]
selected centers: [] 0
Objective Value: 0.0
0.026606281753629446
(30, 150) (30,)
----------
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]

Presolved problem has:
      1046 rows          956 cols         2692 elements       956 globals
Presolve finished in 0 seconds
Heap usage: 6201KB (peak 10MB, 1096KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 64.2GB
 *** Heuristic solution found:    37.000000      Time: 0 ***
 *** Heuristic solution found:    25.000000      Time: 0 ***
 *** Heuristic solution found:    22.000000      Time: 0 ***
Starting concurrent solve with dual

 Concurrent-Solve,   0s
            Dual        
    objective   sum inf 
 P  12.832175   .0000000
------- optimal --------
Concurrent statistics:
      Dual: 960 simplex iterations, 0.01s
Optim

   1  K     22.000000    12.693996      3     12      0   42.30%     824      0
   2  K     22.000000    12.747969      3      7      8   42.05%     802      0
   3  K     22.000000    12.774326      3      6      8   41.93%     781      0
   4  K     22.000000    12.792867      3      6      6   41.85%     777      0
   5  K     22.000000    12.818759      3      5      5   41.73%     761      0
   6  K     22.000000    12.834445      3      4      4   41.66%     778      0
   7  K     22.000000    12.838423      3      3      2   41.64%     792      0
   8  K     22.000000    12.842139      3      3      2   41.63%     725      0
   9  K     22.000000    12.847997      3      4      2   41.60%     738      1
  10  K     22.000000    12.851380      3      3      3   41.58%     769      1
  11  K     22.000000    12.851526      3      2      3   41.58%     788      1
  12  K     22.000000    12.851526      3      0      1   41.58%     797      1
Heuristic search started
Heuristic searc

   2  K     21.000000    11.689545      4      3      2   44.34%    1195      0
   3  K     21.000000    11.702665      4      3      5   44.27%    1214      0
   4  K     21.000000    11.708954      4      3      3   44.24%    1158      1
q           20.000000    11.712855      5                 41.44%       0      1
   5  K     20.000000    11.712855      5      3      2   41.44%    1153      1
   6  K     20.000000    11.714930      5      3      0   41.43%    1090      1
   7  K     20.000000    11.717622      5      1      1   41.41%    1158      2
   8  K     20.000000    11.719969      5      2      2   41.40%    1196      2
   9  K     20.000000    11.721722      5      2      2   41.39%    1152      3
  10  K     20.000000    11.723015      5      2      2   41.38%    1146      4
  11  K     20.000000    11.723015      5      0      1   41.38%    1173      4
Heuristic search started
Heuristic search stopped
 
Cuts in the matrix         : 10
Cut elements in the matrix : 14940
 

    3106    18.000000    12.240590      7     13      6   32.00%     892     29
    3116    18.000000    12.240590      7     20      6   32.00%     787     29
    3126    18.000000    12.240590      7     28      7   32.00%     917     29
    3136    18.000000    12.240590      7     33      8   32.00%     945     29
    3146    18.000000    12.240590      7     38      8   32.00%    1016     30
    3156    18.000000    12.299757      7     46      7   31.67%     718     30
    3166    18.000000    12.299757      7     51      8   31.67%     929     30
    3176    18.000000    12.299757      7     57      8   31.67%     885     30
    3186    18.000000    12.299757      7     66      8   31.67%     943     30
Heap usage: 481MB (peak 481MB, 46MB system)
B&B tree size: 474k total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    3286    18.000000    12.372084      7    162     11   31.27%    1001     31
    3386    18.000000    12.372084      7    262

...

     500    18.000000    14.904270      6    302     15   17.20%     551      4
     602    18.000000    14.904270      6    320     26   17.20%     447      4
     704    18.000000    14.904270      6    387     22   17.20%     354      4
     804    18.000000    14.904270      6    464     10   17.20%     484      4
     907    18.000000    14.929809      6    520     19   17.06%     426      4
    1010    18.000000    14.998719      6    572     16   16.67%     441      4
    1112    18.000000    14.998719      6    591     14   16.67%     513      4
    1214    18.000000    14.998719      6    656     13   16.67%     551      5
    1318    18.000000    14.998719      6    696     15   16.67%     361      5
    1419    18.000000    14.998719      6    736     15   16.67%     520      5
    1521    18.000000    14.998719      6    776     18   16.67%     467      5
    1621    18.000000    14.998719      6    817     15   16.67%     432      5
    1724    18.000000    15.058206      

   1  K     20.000000    11.368646      4      7      0   43.16%    1344      0
   2  K     20.000000    11.384870      4      3      1   43.08%    1349      0
   3  K     20.000000    11.394101      4      4      6   43.03%    1364      0
   4  K     20.000000    11.402961      4      3      3   42.99%    1400      1
   5  K     20.000000    11.407832      4      3      2   42.96%    1440      1
   6  K     20.000000    11.408709      4      4      1   42.96%    1391      2
   7  K     20.000000    11.409152      4      2      1   42.95%    1420      2
   8  K     20.000000    11.409177      4      2      2   42.95%    1437      3
   9  K     20.000000    11.409192      4      1      1   42.95%    1398      4
  10  K     20.000000    11.409192      4      0      0   42.95%    1420      4
Heuristic search started
Heuristic search stopped
 
Cuts in the matrix         : 12
Cut elements in the matrix : 15339
 
Starting tree search.
Deterministic mode with up to 32 running threads and up t

    3279    18.000000    12.053503      6     13      5   33.04%    1127     25
    3289    18.000000    12.053503      6     21      8   33.04%    1168     25
    3299    18.000000    12.053503      6     26      6   33.04%    1162     25
    3309    18.000000    12.053503      6     33      7   33.04%     837     26
    3319    18.000000    12.053503      6     39      7   33.04%    1042     26
    3329    18.000000    12.053503      6     44     10   33.04%    1195     26
    3339    18.000000    12.053503      6     52      8   33.04%     964     26
    3349    18.000000    12.053503      6     58     10   33.04%    1021     26
    3359    18.000000    12.053503      6     68      9   33.04%     884     26
Heap usage: 418MB (peak 453MB, 40MB system)
B&B tree size: 0.8MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    3459    18.000000    12.053503      6    167     12   33.04%    1023     27
    3559    18.000000    12.053503      6    26

   12038    18.000000    12.053503      6   6422     23   33.04%     847     46
   12139    18.000000    12.053503      6   6449     31   33.04%     719     46
   12240    18.000000    12.053503      6   6476     33   33.04%     516     46
   12340    18.000000    12.053503      6   6476     27   33.04%     739     46
   12440    18.000000    12.053503      6   6488     32   33.04%     617     46
   12540    18.000000    12.053503      6   6511     45   33.04%     415     46
   12641    18.000000    12.053503      6   6535     32   33.04%     532     47
   12741    18.000000    12.053503      6   6535     28   33.04%     669     47
   12842    18.000000    12.053503      6   6583     31   33.04%     715     47
   12944    18.000000    12.053503      6   6583     27   33.04%     525     47
   13044    18.000000    12.053503      6   6707     40   33.04%     580     47
   13144    18.000000    12.053503      6   6751     23   33.04%     575     47
   13245    18.000000    12.053503      

Heap usage: 1108MB (peak 1108MB, 75MB system)
B&B tree size: 310MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
   97322    17.000000    12.971090      7  50771     29   23.70%     635    136
   98322    17.000000    12.971090      7  51289     42   23.70%     579    136
   99327    17.000000    12.971090      7  51767     28   23.70%     446    138
  100330    17.000000    12.999081      7  52294     37   23.53%     579    139
  101331    17.000000    12.999081      7  52875     34   23.53%     528    140
  102332    17.000000    13.015570      7  53430     38   23.44%     484    141
  103333    17.000000    13.015570      7  53913     32   23.44%     633    142
  113334    17.000000    13.183268      7  59292     39   22.45%     515    152
  123336    17.000000    13.273449      7  64558     35   21.92%     414    161
  133336    17.000000    13.399089      7  69795     42   21.18%     503    171
  143337    17.000000    13.588921      7  74

#### Add a constraint that at least 80% of nodes to be taken from pruned set

In [2]:
import numpy as np
import sys
import pickle
import re
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from termcolor import colored
import xpress as xp
from sklearn.metrics import confusion_matrix
import random

nV=[150]                      #nv is the number of vertices in the graph.  
nC=[15]                      #cv is the number of clients for each facility
num_graphs=5

problem='set-cover'
modes=['skewed[1-10]']
da=['training','testing']

def kmedian_xpress_solver_constrained(c,n,fac_index):
    x = np.array ([xp.var (vartype = xp.binary) for i in range(n*n)]).reshape (n,n)
    y = np.array([xp.var (vartype = xp.binary) for i in range(n)])

    c2 = [x[i,j] - y[i] <=0 for i in range(n) for j in range(n)]
    c3 = [xp.Sum(xp.Dot(x[:,j],c[:,j])) ==1 for j in range(n)]
    S=range(n)
    fac_inverse = [i for i in S if i not in fac_index]
    print(colored(fac_inverse,'magenta'))
    if len(fac_inverse) > 0:
        c4=[xp.Sum(y[fac_inverse])<=2]
    else:
        c4=[xp.Sum(y[0])<=1]
        
    profit=xp.Sum(y)

    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.minimize)
    try:
        p.addConstraint(c2,c3,c4)#,c4,c5,c6,c7)
    except xp.ModelError as e:
      print ("Modeling error:", repr(e))
    p.solve()
    
  
    

   
    median_indices=np.where(p.getSolution(y)>=0.5)[0]
 
    return median_indices,p.getSolution(profit)


#///////////////////////////////////////////////////////////////////////////////////////////////////
for nv in nV:
    for nc in nC:
        for Mode in modes:        
            path='data/'+problem+'/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/testing/'
            thre = [max(int(nv/10),1),max(int(nv/5),1),max(int(3*nv/10),1),max(int(4*nv/10),1),int(nv/2),int(nv)]
            value_n=np.zeros(len(thre))
            Time=np.zeros(len(thre))
            for m in range(num_graphs):
                print(colored('....................................................','blue'),m)
                set_path=path+str(m)
                data = pickle.load(open(set_path+'/data','rb'))

                probs=pickle.load(open(set_path+'/probs','rb'))
                ind=np.argsort(probs)


                n=data[1];
                data=data[0].astype(int)
                data[:,0]=data[:,0]-1
                data[:,1]=data[:,1]-1

                for th_index,th in enumerate(thre):
                    ind_last_thre = ind[len(ind)-th:]

                    cost_mtr=np.zeros([n,n])
                    for i in range (len(data)):
                        cost_mtr[data[i,0],data[i,1]]=data[i,2]

                    #print(colored("--------------------",'blue'))
                    #print(data,n,c,num_fac,k)
                    #print(colored("--------------------",'blue'))
                    S=timer()
                    median_indices,value=kmedian_xpress_solver_constrained(cost_mtr,n,ind_last_thre)
                    E=timer()
                    print(colored('pruned centers:','red'),np.sort(ind_last_thre)+1)
                    print(colored('selected centers:','red'),np.sort(median_indices)+1,len(median_indices))
                    print(colored('Objective Value:','red'),value)
                    print(E-S)

                    s=pickle.load(open(set_path+'/points_inf_xp','rb'))
                    s=s[0][1]
                    value_n[th_index]=value/s
                    Time[th_index]=E-S
                    #print(s,value_n[thi],value)
                print(colored("Value_n: " +str(value_n),'red'))
                print(colored("Time: "+str(Time),'red'))

.................................................... 0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 22, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 43, 44, 46, 47, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 107, 109, 110, 111, 112, 113, 114, 116, 117, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]
FICO Xpress v8.10.0, Hyper, solve started 22:08:36, Aug 14, 2021
Heap usage: 11MB (peak 11MB, 1316KB system)
Minimizing MILP noname
Original problem has:
     22651 rows        22650 cols        47385 elements     22650 globals
Presolved problem has:
      2401 rows         2400 cols         6885 elements      2400 globals
Presolve finished in 0 seconds
Heap usage: 1

       9    20.000000    17.764728      4      1      6   11.18%     358      3
      10    20.000000    17.979817      4      2      6   10.10%     353      3
 *** Search completed ***
Uncrunching matrix
Final MIP objective                   : 1.999999999999999e+01
Final MIP bound                       : 1.999999999999999e+01
  Solution time / primaldual integral :         3s/ 20.965745%
  Number of solutions found / nodes   :         4 /        11
  Max primal violation      (abs/rel) : 2.005e-13 / 2.005e-13
  Max integer violation     (abs    ) : 2.005e-13

pruned centers: [  6  15  18  20  22  24  28  30  37  43  46  49  50  57  58  59  74  87
  96 103 107 109 115 116 119 120 128 142 146 147]
selected centers: [ 15  20  22  30  43  58  59  74  87  91  92  96 103 116 119 120 128 142
 146 147] 20
Objective Value: 19.99999999999999
3.7315431269817054
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 18, 20, 22, 24, 25, 28, 30, 31, 32, 33, 34, 35, 37, 38, 39, 43, 44, 46, 47, 50, 51, 

    1554    16.000000    13.253736      9    243     17   17.16%     626     15
    1656    16.000000    13.257344      9    237     13   17.14%     749     15
    1756    16.000000    13.257344      9    213     18   17.14%     595     15
    1858    16.000000    13.257344      9    169     20   17.14%     588     15
    1958    16.000000    13.257344      9    171     14   17.14%     554     15
Heap usage: 812MB (peak 812MB, 72MB system)
B&B tree size: 5.9MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    2059    16.000000    13.257344      9    163     18   17.14%     627     16
    2160    16.000000    13.257344      9    167     16   17.14%     618     16
    2260    16.000000    13.257344      9    177     19   17.14%     682     16
    2360    16.000000    13.263205      9    188     16   17.10%     604     16
    2460    16.000000    13.263205      9    188     14   17.10%     613     16
    2560    16.000000    13.263205      9    18

Heap usage: 249MB (peak 249MB, 20MB system)
B&B tree size: 1.2MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
     200    20.000000    12.809579      4    184     18   35.95%     789     11
     300    20.000000    12.809579      4    284     23   35.95%     662     11
q    325    19.000000    12.809579      5    310     25   32.58%       0     12
     400    19.000000    12.809579      5    384     34   32.58%     567     12
     500    19.000000    12.809579      5    484     32   32.58%     699     12
     600    19.000000    12.809579      5    584     40   32.58%     641     13
     700    19.000000    12.809579      5    678     39   32.58%     627     13
c    722    18.000000    12.809579      6    697     56   28.84%       0     13
     800    18.000000    12.809579      6    647     46   28.84%     530     13
     900    18.000000    12.809579      6    668     12   28.84%     760     14
    1001    18.000000    12.809579      6    68

    6184    17.000000    12.865999      7     14      7   24.32%     904     28
    6194    17.000000    12.865999      7     19      6   24.32%     953     28
    6204    17.000000    12.865999      7     26      8   24.32%     820     28
    6214    17.000000    12.865999      7     31      8   24.32%     752     29
    6224    17.000000    12.865999      7     40      8   24.32%     901     29
    6234    17.000000    12.865999      7     45      9   24.32%     713     29
    6244    17.000000    12.865999      7     52      9   24.32%     821     29
    6254    17.000000    12.865999      7     59      9   24.32%     815     29
    6264    17.000000    12.865999      7     66      7   24.32%     834     29
Heap usage: 885MB (peak 931MB, 69MB system)
B&B tree size: 0.9MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    6364    17.000000    12.865999      7    126      9   24.32%     743     30
    6464    17.000000    12.889513      7    21

...

   30167    17.000000    13.363452      7   7009     23   21.39%     514     71
   31170    17.000000    13.389984      7   7190     17   21.24%     638     72
   32170    17.000000    13.389984      7   7342     16   21.24%     770     73
   33171    17.000000    13.520245      7   7505     17   20.47%     585     74
   34174    17.000000    13.520245      7   7613     19   20.47%     443     75
   35174    17.000000    13.520245      7   7736     26   20.47%     643     76
   36174    17.000000    13.561958      7   7821     20   20.22%     634     77
   37177    17.000000    13.561958      7   7936     16   20.22%     467     77
   38181    17.000000    13.561958      7   8052     21   20.22%     533     78
   39181    17.000000    13.567110      7   8194     13   20.19%     782     79
   40183    17.000000    13.567110      7   8326     24   20.19%     618     80
   41188    17.000000    13.579416      7   8434     21   20.12%     488     81
   42191    17.000000    13.670520      

   1  K     21.000000    11.368646      4      7      0   45.86%    1338      0
   2  K     21.000000    11.384870      4      3      1   45.79%    1351      0
   3  K     21.000000    11.394101      4      4      6   45.74%    1363      0
   4  K     21.000000    11.402961      4      3      3   45.70%    1409      1
   5  K     21.000000    11.407832      4      3      2   45.68%    1444      1
   6  K     21.000000    11.408709      4      4      1   45.67%    1401      2
   7  K     21.000000    11.409152      4      2      1   45.67%    1444      2
   8  K     21.000000    11.409177      4      2      2   45.67%    1444      3
   9  K     21.000000    11.409192      4      1      1   45.67%    1415      4
  10  K     21.000000    11.409192      4      0      0   45.67%    1429      4
Heuristic search started
Heuristic search stopped
M           19.000000    11.409192      5                 39.95%       0      4
 
Cuts in the matrix         : 12
Cut elements in the matrix : 15345
 

    3189    19.000000    12.016396      5     14      6   36.76%    1173     33
    3199    19.000000    12.016396      5     19      6   36.76%    1187     34
    3209    19.000000    12.016396      5     26      8   36.76%     967     34
    3219    19.000000    12.016396      5     33      6   36.76%     880     34
    3229    19.000000    12.016396      5     39      9   36.76%     934     34
    3239    19.000000    12.016396      5     45      8   36.76%    1112     34
    3249    19.000000    12.016396      5     53      7   36.76%     808     34
    3259    19.000000    12.016396      5     60      8   36.76%    1073     34
    3269    19.000000    12.016396      5     67      9   36.76%     810     34
Heap usage: 496MB (peak 496MB, 41MB system)
B&B tree size: 416k total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    3369    19.000000    12.016396      5    166     14   36.76%     947     35
    3469    19.000000    12.016396      5    266

   13140    17.000000    12.016396      7   4442     41   29.32%     435     53
   13247    17.000000    12.016396      7   4459     52   29.32%     369     54
   14247    17.000000    12.016396      7   4583     25   29.32%     501     55
   15249    17.000000    12.016396      7   5370     34   29.32%     683     56
   16255    17.000000    12.016396      7   5968     51   29.32%     523     57
   17257    17.000000    12.016396      7   6269     44   29.32%     599     59
   18261    17.000000    12.016396      7   7173     51   29.32%     552     60
   19263    17.000000    12.016396      7   7581     28   29.32%     817     61
   20264    17.000000    12.016396      7   8251     34   29.32%     785     62
   21266    17.000000    12.016396      7   8912     30   29.32%     711     63
   22266    17.000000    12.016396      7   9469     32   29.32%     718     65
   23266    17.000000    12.016396      7  10177     11   29.32%     971     66
   24266    17.000000    12.016396      

Heap usage: 1408MB (peak 1408MB, 75MB system)
B&B tree size: 470MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
  143397    17.000000    13.582164      7  76950     39   20.10%     662    192
  153397    17.000000    13.605386      7  82126     34   19.97%     777    201
  163397    17.000000    13.627052      7  87099     28   19.84%     906    211
  173402    17.000000    13.659104      7  92225     35   19.65%     541    221
  183402    17.000000    13.682441      7  97188     30   19.52%     426    230
  193403    17.000000    13.723605      7 102101     27   19.27%     749    240
  203403    17.000000    13.729342      7 107051     33   19.24%     498    250
  213404    17.000000    13.756617      7 111930     34   19.08%     705    259
  223406    17.000000    13.762905      7 116728     46   19.04%     572    269
  233407    17.000000    13.792811      7 121453     34   18.87%     484    279
  243410    17.000000    13.812442      7 126